# Importation

In [1]:
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download -d enzodurand/boudingboxonlyhanddataset
# ! unzip boudingboxonlyhanddataset.zip

In [2]:
import os
import copy
import cv2
import wandb
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import time
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score 
from matplotlib import pyplot as plt

import torchvision
from torchvision import models, transforms
from torchvision.io import read_image

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset

# !pip install albumentations
import albumentations as A

# GPU/TPU setup

In [3]:
## TPU
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()
# torch.set_default_tensor_type('torch.FloatTensor')

## GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## Weight and biases
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


cuda:0


wandb: Currently logged in as: hanzopgp (use `wandb login --relogin` to force relogin)


True

# Global variables 

In [4]:
INPUT_SIZE = 400
N_CLASS = 4
WHERE = "home"

In [5]:
if WHERE=="colab":
	PATH_LABELS = "/content/index_label_bbox.csv"
	PATH_IMG = "/content/output/output"
	PATH_LABELS_VALID = "/content/index_label_bbox_validation.csv"
	PATH_IMG_VALID = "/content/output_validation/output_validation"
	BATCH_SIZE = 64
elif WHERE=="kaggle":
	PATH_LABELS = "../input/boudingboxonlyhanddataset/index_label_bbox.csv"
	PATH_IMG = "../input/boudingboxonlyhanddataset/output/output"
	PATH_LABELS_VALID = "../input/boudingboxonlyhanddataset/index_label_bbox_validation.csv"
	PATH_IMG_VALID = "../input/boudingboxonlyhanddataset/output_validation/output_validation"
	BATCH_SIZE = 64
elif WHERE=="home":
	PATH_LABELS = "../../../data_labels/bounding_box_model/done/index_label_bbox.csv"
	PATH_IMG = "../../../data_labels/bounding_box_model/done/output"
	PATH_LABELS_VALID = "../../../data_labels/bounding_box_model/done_validation/index_label_bbox_validation.csv"
	PATH_IMG_VALID = "../../../data_labels/bounding_box_model/done_validation/output_validation"
	BATCH_SIZE = 4

# Data functions

In [6]:
class HandGestureDataset(Dataset):
	def __init__(self, annotations_file, img_dir, transform=None):
		self.img_labels = pd.read_csv(annotations_file)
		self.img_dir = img_dir
		self.transform = transform

	def __len__(self):
		return len(self.img_labels)

	def __getitem__(self, idx):
		img_path = os.path.join(self.img_dir, os.listdir(self.img_dir)[idx])
		image = read_image(img_path)
		# image = image/INPUT_SIZE
		path = str("output/"+os.listdir(self.img_dir)[idx]).split("/")[0]
		line = self.img_labels["index"] == str(path+"/"+os.listdir(self.img_dir)[idx])
		x, y, x_end, y_end = self.img_labels.loc[line]["x"].item(),\
								self.img_labels.loc[line]["y"].item(),\
								self.img_labels.loc[line]["x_end"].item(),\
								self.img_labels.loc[line]["y_end"].item()
		# x, y, x_end, y_end = x/INPUT_SIZE, y/INPUT_SIZE, x_end/INPUT_SIZE, y_end/INPUT_SIZE
		image = image.permute(1,2,0)
		# image = image/INPUT_SIZE
		# print("beforetransfo", x, y, x_end, y_end)
		if self.transform:
			transformed = self.transform(image=np.array(image), bboxes=[[x,y,x_end,y_end]])
		transformed_image = transformed['image']
		transformed_bboxes = transformed['bboxes']
		# print(transformed_image)
		# print(transformed_bboxes)
		# return {'image':image, 'x':x, 'y':y, 'x_end':x_end, 'y_end':y_end}
		# return {'image':image, 'bboxes':[x,y,x_end,y_end]}
		# print("transfo",transformed_bboxes)
		return transformed_image, transformed_bboxes

In [7]:
def draw_predictions(image, preds):
	startX, startY, endX, endY = preds
	# scale the predicted bounding box coordinates based on the image
	# dimensions    
	startX = int(startX * INPUT_SIZE)
	startY = int(startY * INPUT_SIZE)
	endX = int(endX * INPUT_SIZE)
	endY = int(endY * INPUT_SIZE)
	#     print(startX, startY, endX, endY)
	# draw the predicted bounding box on the image
	image = image.numpy().copy()
	cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
	# show the output image
	plt.imshow(image)
	plt.show()

def prepare_data_vgg(data_type):
    ## Parameters fitting vgg/imagenet
	mean=[0.485, 0.456, 0.406]
	std=[0.229, 0.224, 0.225]

	transformVGGTrainAlbu = A.Compose([
		# A.RandomCrop(width=400, height=400),
		# A.HorizontalFlip(p=0.5),
		A.RandomBrightnessContrast(p=0.5),
	], bbox_params=A.BboxParams(format='pascal_voc', label_fields=""))

	## pytorch transformer objects
	transformVGGTrain=transforms.Compose([
			transforms.ToPILImage(),
			## can try
			# transforms.ColorJitter(brightness=0.5, hue=0.5),
			# transforms.RandomAdjustSharpness(sharpness_factor=3),
			# transforms.RandomAutocontrast(),
			# transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.01, 5)),
			## don't use or it will ruin bounding box labels
	#             transforms.RandomRotation(degrees=(-15, 15)),
	#             transforms.RandomPerspective(distortion_scale=0.4, p=0.2),
	#             transforms.RandomAffine(degrees=(0, 5), translate=(0, 0.18), scale=(0.7, 1)),
			## too much
			# transforms.RandomSolarize(threshold=192.0),
	#             transforms.RandomPosterize(bits=2),
	#             transforms.RandomInvert(),
	#             transforms.RandomEqualize(),
			transforms.Resize(size=(INPUT_SIZE, INPUT_SIZE)),
			transforms.ToTensor(),
	#             transforms.Normalize(mean, std) ## test with and without
        ])
	transformVGGValid=transforms.Compose([
			transforms.ToPILImage(),
			transforms.Resize(size=(INPUT_SIZE, INPUT_SIZE)),
			transforms.ToTensor(),
	#             transforms.Normalize(mean, std) ## test with and without
		])

	if data_type == "custom":
		## Custom dataset
		VGG_dataset_train = HandGestureDataset(PATH_LABELS, PATH_IMG, transformVGGTrainAlbu)
		VGG_dataset_valid = HandGestureDataset(PATH_LABELS_VALID, PATH_IMG_VALID, transformVGGValid)
		VGG_trainloader = torch.utils.data.DataLoader(VGG_dataset_train, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)
		VGG_validloader = torch.utils.data.DataLoader(VGG_dataset_valid, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)

	return VGG_trainloader, VGG_validloader

# Loading data into pytorch dataset and dataloader objects

In [8]:
VGG_trainloader, VGG_validloader = prepare_data_vgg("custom")

In [9]:
for batch in VGG_trainloader:
	img, bbox = batch
	res = []
	for e in bbox:
		res.append(np.array(e))
	res = np.array(res).T
	for i, l in zip(img, res):
		print(i.shape)
		print(l)
		break
	break
		# draw_predictions(i, l.item())
	# 	break
	# break
	# 	break
	# break

# def show_img(img):
#     plt.figure(figsize=(18,15))
#     # unnormalize
#     img = img / 2 + 0.5  
#     npimg = img.numpy()
#     npimg = np.clip(npimg, 0., 1.)
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()
# data = iter(VGG_trainloader)
# images = data.next()
# show_img(torchvision.utils.make_grid(images))

torch.Size([400, 400, 3])
[tensor([ 17.,  76., 170., 119.], dtype=torch.float64)]


C:\Users\karna\AppData\Local\Temp\ipykernel_22644\500527189.py:5: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  res.append(np.array(e))
C:\Users\karna\AppData\Local\Temp\ipykernel_22644\500527189.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res.append(np.array(e))


# Model functions

In [10]:
def train(model, epochs, train_loader, valid_loader, learning_rate, patience, feature_extract=False):
	## Early stopping variables
	es = EarlyStopping(patience=patience)
	terminate_training = False
	best_model_wts = copy.deepcopy(model.state_dict())
	best_loss = np.inf
	model = model.to(device)
	## Training only the parameters where we require gradient since we are fine-tuning
	params_to_update = model.parameters()
	print("params to learn:")
	if feature_extract:
		params_to_update = []
		for name,param in model.named_parameters():
			if param.requires_grad == True:
				params_to_update.append(param)
				print("\t", name)
	else:
		for name,param in model.named_parameters():
			if param.requires_grad == True:
				print("\t", name)
                
	## Setting up our optimizer
	optim = torch.optim.Adam(params_to_update, lr=learning_rate)

	## Setting up our loss function
	loss = nn.MSELoss()
    
	## Running the train loop
	print(f"running {model.name}")
	for epoch in range(epochs):
		cumloss, count = 0, 0
		model.train()
		for item in train_loader:
			optim.zero_grad()
			x, y = item["image"], item["label"]
			x = item["image"].to(device)
			
			res = []
			for e in y:
				res.append(np.array(e))
			res = np.array(res).T
			y = torch.as_tensor(res)
			y = y.to(torch.float32)
			y = y.to(device)
			
			yhat = model(x)

			l = loss(yhat, y)
			l.backward()
			# xm.optimizer_step(optim, barrier=True)
			optim.step()
			cumloss += l * len(x)
			count += len(x)
		print("epoch :", epoch, end="")
		loss_ = cumloss.cpu().item()/count
		wandb.log({'train_loss': loss_})
		print(", train_loss: ", loss_, end="")
		if epoch % 1 == 0:
			model.eval()
			with torch.no_grad():
				valid_cumloss, count = 0, 0
				for x,y in valid_loader:
					x, y = item["image"], item["label"]
					x = item["image"].to(device)
					res = []
					for e in y:
						res.append(np.array(e))
					res = np.array(res).T
					y = torch.as_tensor(res)
					y = y.to(torch.float32)
					y = y.to(device)
					yhat = model(x)
					valid_cumloss += loss(yhat,y) * len(x)
					count += len(x)
				valid_loss_ = valid_cumloss.cpu().item()/count
				wandb.log({'valid_loss': valid_loss_})
				print(", valid_loss: ", valid_loss_)
				## Early stopping
				if valid_cumloss/count < best_loss:
					best_loss = valid_cumloss/count
					best_model_wts = copy.deepcopy(model.state_dict())
				if es.step(valid_cumloss.cpu().item()/count):
					terminate_training = True
					break
		if terminate_training:
			break
	print('Best val loss: {:4f}'.format(best_loss))
	## Returns the best model
	model.load_state_dict(best_model_wts)
	return model

def set_parameter_requires_grad(model, feature_extract):
	if feature_extract:
		for name,p in model.named_parameters():
			if "features" in name:
				p.requires_grad = False    
			else:
				p.requires_grad = True  

# Loading the model and modifying the classifier part
### Maybe we could try to modify only the last classifier layer ?

In [11]:
## Loading vgg16 model pretrained on imagenet
vgg = models.vgg16(pretrained=True)

vgg.classifier = nn.Sequential(nn.Linear(25088, 4096), 
                               nn.ReLU(), 
                              #  nn.Dropout(0.5),        
                               nn.Linear(4096, 1024), 
                               nn.ReLU(), 
#                                nn.Dropout(0.5),        
                               nn.Linear(1024, 256),
                               nn.ReLU(), 
#                                nn.Dropout(0.5),        
                               nn.Linear(256, N_CLASS),
                               nn.Sigmoid())

print(vgg.eval())

## Sets all the requires grad of the classifier layers to True
set_parameter_requires_grad(vgg, True)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Implementing early stopping

In [12]:
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)
        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False
        if np.isnan(metrics):
            return True
        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1
        if self.num_bad_epochs >= self.patience:
            return True
        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

# Training only the modified parts of the classifier

In [13]:
os.environ['WANDB_NOTEBOOK_NAME'] = '4096_5e-6'
wandb.init(project="jetson-autonomous-driving")

In [ ]:
print(len(VGG_trainloader))
print(len(VGG_validloader))

## Fine-tuning the model on our data
vgg.name = "VGG"

best_model = train(model=vgg, 
                   epochs=1000, 
                   train_loader=VGG_trainloader, 
                   valid_loader=VGG_validloader, 
                   learning_rate=5e-6, ## 3e-4 learning rate for Adam optimizer
                   patience=20) ## metric for earlystopping : val_loss 

536
151
params to learn:
	 classifier.0.weight
	 classifier.0.bias
	 classifier.2.weight
	 classifier.2.bias
	 classifier.4.weight
	 classifier.4.bias
	 classifier.6.weight
	 classifier.6.bias
running VGG


RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same

# Checking predictions

In [ ]:
# with torch.no_grad():
#     for item in VGG_validloader:
#         x, y = item["image"], item["label"]
#         base_img = item["image"]
#         x = item["image"].to(device)
#         res = []
#         for e in y:
#             res.append(np.array(e))
#         res = np.array(res).T
#         y = torch.as_tensor(res)
#         y = y.to(torch.float32)
#         y = y.to(device)
#         yhat = best_model(x)
#         for i in range(4):
#             draw_predictions(base_img[i].permute(1,2,0), yhat.cpu()[i])

# Saving the model in .pth and .onnx extension

In [ ]:
PATH = "./"
torch.save(best_model.state_dict(), os.path.join(PATH,"boundingbox_vgg_last.pth"))
# from google.colab import files
# files.download(os.path.join(PATH,"boundingbox_vgg_last.pth"))

In [ ]:
# del vgg
# del best_model

: 

In [ ]:
# model = models.vgg16(pretrained=True)
# model.classifier[0] = nn.Linear(25088, 8192)
# model.classifier[3] = nn.Linear(8192, 1024)
# model.classifier[6] = nn.Linear(1024, N_CLASS)
# model.load_state_dict(torch.load(os.path.join(PATH,"vgg.pth"), map_location='cpu'))
# model.eval() 

# dummy_input = torch.randn(BATCH_SIZE, 3, INPUT_SIZE, INPUT_SIZE)  
# torch.onnx.export(model,   
#                   dummy_input, 
#                   "vgg.onnx",
#                   export_params=True,
#                   do_constant_folding=True, 
#                   input_names = ['modelInput'],
#                   output_names = ['modelOutput'])

: 

: 

: 